**TODO**

* figure out how to use RigidBodyDynamics

In [2]:
using RigidBodyDynamics
using StaticArrays

In [3]:
g = -9.81 # gravitational acceleration in z-direction
world = RigidBody{Float64}("world")
twolinkarm = Mechanism(world; gravity = SVector(0, 0, g))

Spanning tree:
Vertex: world (root)
No non-tree joints.

In [4]:
axis = SVector(0., 0., 1.) # joint axis
I_1 = 0.333 # moment of inertia about joint axis
c_1 = -0.5 # center of mass location with respect to joint axis
m_1 = 1. # mass
frame1 = CartesianFrame3D("upper_link") # the reference frame in which the spatial inertia will be expressed
inertia1 = SpatialInertia(frame1, I_1 * axis * axis.', m_1 * SVector(0, 0, c_1), m_1)

SpatialInertia expressed in "upper_link":
mass: 1.0
center of mass: Point3D in "upper_link": [0.0, 0.0, -0.5]
moment of inertia:
[0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.333]

In [5]:
upperlink = RigidBody(inertia1)

RigidBody: "upper_link"

In [6]:
shoulder = Joint("shoulder", Revolute(axis))

Joint "shoulder": Revolute joint with axis [0.0, 0.0, 1.0]

In [7]:
beforeShoulderToWorld = eye(Transform3D, frame_before(shoulder), default_frame(world))

Transform3D from "before_shoulder" to "world":
rotation: 0.0 rad about [1.0, 0.0, 0.0], translation: [0.0, 0.0, 0.0]

In [9]:
attach!(twolinkarm, world, shoulder, beforeShoulderToWorld, upperlink)

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
No non-tree joints.

In [30]:
l_1 = -1. # length of the upper link
I_2 = 0.333 # moment of inertia about joint axis
c_2 = -0.5 # center of mass location with respect to joint axis
m_2 = 1. # mass
inertia2 = SpatialInertia(CartesianFrame3D("lower_link"), I_2 * axis * axis.', m_2 * SVector(0, 0, c_2), m_2)
lowerlink = RigidBody(inertia2)
elbow = Joint("elbow", Revolute(axis))
beforeElbowToAfterShoulder = Transform3D(frame_before(elbow), shoulder.frameAfter, SVector(0, 0, l_1))
attach!(twolinkarm, upperlink, elbow, beforeElbowToAfterShoulder, lowerlink)

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
    Vertex: lower_link, Edge: elbow
No non-tree joints.

In [31]:
state = MechanismState{Float64}(twolinkarm)
configuration(state, shoulder)[:] = 0.0
configuration(state, elbow)[:] = 0.0
velocity(state, shoulder)[:] = 0.0
velocity(state, elbow)[:] = 0.0;

setdirty!(state)

In [32]:
ts, qs, vs = simulate(state, 0.1, Δt = 1e-2);